In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import astropy
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord
import glob

In [16]:
# Load all mask files and combine
mask_files = sorted(glob.glob('masks/*/*_slits.csv'))

mask_df = {}
for i,mask_file in enumerate(mask_files):
    mask_name = mask_file.split('BLAS_')[-1].split('_slits.csv')[0]
    if mask_name == '1':
        mask_name = '2019'
    mask_df[i] = pd.read_csv(mask_file)
    mask_df[i]['mask'] = mask_name
    if i>0:
        mask_df[0] = mask_df[0].append(mask_df[i])
        
mask_all = mask_df[0].reset_index()

In [89]:
mask_all

,index,slit,targetnum,name,ra,dec,pmra,pmdec,mag,priority,epoch,type,wave_start,wave_end,side,mask
0,0,22,778,3DHST_15563,188.950745,62.218903,0,0,0.001,6.000000,2000,TARGET,7728.533648,10251.126548,A,2019
1,1,23,818,3DHST_16476,188.950455,62.223740,0,0,0.001,6.000000,2000,TARGET,7698.929822,10221.535552,A,2019
2,2,27,872,3DHST_17848,188.946426,62.230289,0,0,0.001,6.000000,2000,TARGET,7652.275725,10174.901935,A,2019
3,3,33,908,3DHST_19025,188.946198,62.235584,0,0,0.001,6.000000,2000,TARGET,7620.071986,10142.712164,A,2019
4,4,37,931,3DHST_19661,188.949158,62.238186,0,0,0.001,6.000000,2000,TARGET,7609.846845,10132.491259,A,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,103,391,578,3DHST_28986,189.507950,62.284489,0,0,0.001,634.999939,2000,TARGET,6891.368645,9414.334259,B,2020A_4
752,104,217,326,3DHST_31384,189.473755,62.298164,0,0,0.001,313.000000,2000,TARGET,7041.775641,9564.674574,B,2020A_4
753,105,119,176,3DHST_29862,189.509766,62.289780,0,0,0.001,53.000000,2000,TARGET,6909.809066,9432.767120,B,2020A_4
754,106,691,1008,3DHST_31164,189.497345,62.296669,0,0,0.001,1136.000000,2000,TARGET,6972.917516,9495.847799,B,2020A_4


In [99]:
mask_all[mask_all.duplicated(subset='name')]
mask_all[mask_all.name == 'z8_GNW_27475']

,index,slit,targetnum,name,ra,dec,pmra,pmdec,mag,priority,epoch,type,wave_start,wave_end,side,mask
338,164,116,259,z8_GNW_27475,189.37381,62.336391,0,0,0.001,3.999878,2000,TARGET,7482.474395,10005.177099,B,2020A_1
339,165,125,283,z8_GNW_27475,189.37381,62.336391,0,0,0.001,3.999878,2000,TARGET,7482.474395,10005.177099,B,2020A_1


# High z parent catalogs

In [138]:
parent_cats = sorted(glob.glob('target_lists/pri*.*'))
print(parent_cats)
match_parent_mask = {}
for i, parent_cat in enumerate(parent_cats):
    if i in [0,2,6]:
        skiprows = 1
    else:
        skiprows = 0
    
    delim_whitespace = True
    if i == 1:
        delim_whitespace = False
        
    cat = pd.read_csv(parent_cat, delim_whitespace=delim_whitespace, skiprows=skiprows)
    print(i, parent_cat)
    if i == 4:
        cat[cat.columns[0]] = [f'3DHST_{ID:.0f}' for ID in cat[cat.columns[0]]]
    if i == 6:
        cat[cat.columns[0]] = [f'3DHST_Q_{ID:.0f}' for ID in cat[cat.columns[0]]]

    match_parent_mask[i] = mask_all.merge(cat, right_on=cat.columns[0], left_on='name')
    
    print('Parents:',len(cat))
    print('Matched:',len(match_parent_mask[i]))
    

['target_lists/pri1_knownLAEs_cat_matchFinkelstein.dat', 'target_lists/pri1_standardstars.csv', 'target_lists/pri2_MgII_cat_match3DHST.dat', 'target_lists/pri3_high_z_cat_A.dat', 'target_lists/pri4_filler_cat.dat', 'target_lists/pri5_high_z_cat_B.dat', 'target_lists/pri6_Sirio_binospec_fillers.txt']
0 target_lists/pri1_knownLAEs_cat_matchFinkelstein.dat
Parents: 25
Matched: 25
1 target_lists/pri1_standardstars.csv
Parents: 4
Matched: 10
2 target_lists/pri2_MgII_cat_match3DHST.dat
Parents: 5
Matched: 4
3 target_lists/pri3_high_z_cat_A.dat
Parents: 122
Matched: 122
4 target_lists/pri4_filler_cat.dat
Parents: 1520
Matched: 374
5 target_lists/pri5_high_z_cat_B.dat
Parents: 240
Matched: 217
6 target_lists/pri6_Sirio_binospec_fillers.txt
Parents: 96
Matched: 16


In [139]:
match_parent_mask[1]

,index,slit,targetnum,name,ra_x,dec_x,pmra,pmdec,mag,priority_x,epoch,type_x,wave_start,wave_end,side,mask,ra_y,dec_y,priority_y,type_y
0,19,255,1916,f_3,188.910191,62.352588,0,0,0.001,1.0,2000,STANDARD,6855.558347,9378.533932,A,2019,188.910191,62.352588,1,standard
1,70,2,4,f_3,188.910191,62.352588,0,0,0.001,1.0,2000,STANDARD,6841.397951,9364.381562,A,2020A_1,188.910191,62.352588,1,standard
2,10,2,4,f_3,188.910191,62.352588,0,0,0.001,1.0,2000,STANDARD,7036.745254,9559.638787,A,2020A_4,188.910191,62.352588,1,standard
3,20,254,1915,f_2,188.902346,62.363999,0,0,0.001,1.0,2000,STANDARD,6773.112436,9296.124509,A,2019,188.902346,62.363999,1,standard
4,72,1,3,f_2,188.902346,62.363999,0,0,0.001,1.0,2000,STANDARD,6797.170757,9320.174983,A,2020A_1,188.902346,62.363999,1,standard
5,11,1,3,f_2,188.902346,62.363999,0,0,0.001,1.0,2000,STANDARD,6968.799027,9491.722902,A,2020A_4,188.902346,62.363999,1,standard
6,85,2,2,f_1,189.462416,62.363585,0,0,0.001,1.0,2000,STANDARD,6913.536511,9436.487893,B,2020A_2,189.462416,62.363585,1,standard
7,150,2,2,f_1,189.462416,62.363585,0,0,0.001,1.0,2000,STANDARD,7330.799680,9853.570196,B,2020A_3,189.462416,62.363585,1,standard
8,87,1,1,f_0,189.453601,62.369844,0,0,0.001,1.0,2000,STANDARD,6871.228510,9394.198333,B,2020A_2,189.453601,62.369844,1,standard
9,152,1,1,f_0,189.453601,62.369844,0,0,0.001,1.0,2000,STANDARD,7368.566188,9891.320664,B,2020A_3,189.453601,62.369844,1,standard


In [153]:
master_cat = match_parent_mask[0]
for i, parent_cat in enumerate(parent_cats):
    if i>0:
        master_cat = master_cat.append(match_parent_mask[i])
        
master_cat = master_cat.sort_values(['mask']).reset_index()
master_cat = master_cat[master_cat.columns[2:]]
master_cat

,slit,targetnum,name,ra_x,dec_x,pmra,pmdec,mag,priority,epoch,...,zspec,flux_Lya,EW_Lya,FWHM_kms_Lya,z_bouw,Hmag,photoz,Jmag,id,lmass
0,3,10,Stark11_43_3982,189.039261,62.247654,0,0,0.001,1.0,2000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,155,1556,3DHST_34718,189.165741,62.322956,0,0,0.001,6.0,2000,...,NaN,NaN,NaN,NaN,NaN,NaN,0.7913,23.291707,NaN,NaN
2,128,1482,3DHST_32698,189.195862,62.306946,0,0,0.001,6.0,2000,...,NaN,NaN,NaN,NaN,NaN,NaN,0.9067,25.294130,NaN,NaN
3,187,1614,3DHST_36327,189.198807,62.336708,0,0,0.001,6.0,2000,...,NaN,NaN,NaN,NaN,NaN,NaN,0.9043,24.478091,NaN,NaN
4,196,1628,3DHST_36728,189.197250,62.341419,0,0,0.001,6.0,2000,...,NaN,NaN,NaN,NaN,NaN,NaN,0.8072,24.056142,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,215,323,3DHST_2834,189.166351,62.146820,0,0,0.001,308.0,2000,...,NaN,NaN,NaN,NaN,NaN,NaN,0.8894,24.511150,NaN,NaN
764,196,291,3DHST_3872,189.124527,62.154549,0,0,0.001,264.0,2000,...,NaN,NaN,NaN,NaN,NaN,NaN,0.9068,24.419289,NaN,NaN
765,253,380,3DHST_3398,189.124207,62.150967,0,0,0.001,386.0,2000,...,NaN,NaN,NaN,NaN,NaN,NaN,0.9042,24.675534,NaN,NaN
766,311,458,3DHST_10096,189.068359,62.192924,0,0,0.001,483.0,2000,...,NaN,NaN,NaN,NaN,NaN,NaN,0.9246,24.837545,NaN,NaN


In [154]:
master_cat_nodup = master_cat.drop_duplicates(['slit','name','mask']).reset_index()
master_cat_nodup = master_cat_nodup.rename(columns={"ra_x": "ra", "dec_x": "dec"})
master_cat_nodup.columns


Index(['index', 'slit', 'targetnum', 'name', 'ra', 'dec', 'pmra', 'pmdec',
       'mag', 'priority', 'epoch', 'type', 'wave_start', 'wave_end', 'side',
       'mask', 'ID', 'ra_y', 'dec_y', 'ID_Fink', 'z_Lya', 'z_fink',
       'Reference', 'priority_x', 'type_x', 'priority_y', 'type_y', 'ID_bino',
       'ID_3DHST', 'zspec', 'flux_Lya', 'EW_Lya', 'FWHM_kms_Lya', 'z_bouw',
       'Hmag', 'photoz', 'Jmag', 'id', 'lmass'],
      dtype='object')

In [155]:
master_cat_final = master_cat_nodup[['index', 'slit', 'targetnum', 'name', 'ra', 'dec', 'pmra', 'pmdec',
                                   'mag', 'priority', 'epoch', 'type', 'wave_start', 'wave_end', 'side', 'mask', 
                                  'ID', 'ID_Fink', 'ID_bino', 'ID_3DHST', 'id',
                                  'zspec','z_Lya', 'z_fink', 'z_bouw', 'photoz', 
                                  'flux_Lya', 'EW_Lya', 'FWHM_kms_Lya',
                                   'Hmag', 'Jmag', 'lmass','Reference']]

master_cat_final.to_csv('master_masks_2019-2020_cat.csv', index=False)